In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_excel('inspectionData_04082020.xlsx')

In [3]:
df.dtypes

bu                                               int64
structureID                                     object
area                                            object
region                                          object
siteName                                        object
structType                                      object
ACQ_Port                                        object
addressLine1                                    object
addressLine2                                    object
city                                            object
state                                           object
postcode                                        object
lit                                             object
lat                                            float64
long                                           float64
buType                                          object
owned_or_managed                                object
inspectMaintainReq                              object
groundElev

In [4]:
df['bu'] = df['bu'].astype(str)
df['lat'] = df['lat'].astype(str)
df['long'] = df['long'].astype(str)

In [5]:
df = df.replace('-','NaN')

In [6]:
df['GBI_Team'] = 'NaN'

In [7]:
special_acq_ports = ['ATT', 'BMI', 'BST', 'DCS']

In [8]:
df.head()

,bu,structureID,area,region,siteName,structType,ACQ_Port,addressLine1,addressLine2,city,...,pre-ModSiteWalk_CM_or_FOT,towerMaintenance,vegetation1,vegetation2,TIA_ClimbedInspection,constructionGC1,constructionGC2,constructionGC3,towerModificationVendor_or_Inspector,GBI_Team
0,805761,A,STA,Texas,TX 143 PLANO,GUYED,ABN,670 Fulgham Rd.,NaN,Plano,...,NaN,2020-02-24 00:00:00,2020-04-09 00:00:00,2020-09-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,805762,A,STA,Texas,TX 144 FORNEY,GUYED,ABN,10655 FM 740,NaN,Forney,...,NaN,2020-04-04 00:00:00,2020-04-04 00:00:00,2020-10-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,805763,A,STA,Texas,TX 145 SHERMAN,GUYED,ABN,Dripping Springs Road,NaN,Denison,...,NaN,2020-10-31 00:00:00,2020-04-06 00:00:00,2020-09-21 00:00:00,NaN,2020-11-30 00:00:00,NaN,NaN,NaN,NaN
3,805764,A,STA,Texas,TX 146 MCKINNEY,GUYED,ABN,5259 Alma,NaN,McKinney,...,2020-12-01 00:00:00,2020-08-07 00:00:00,2020-04-19 00:00:00,2020-10-30 00:00:00,NaN,2020-04-11 00:00:00,2020-11-02 00:00:00,2020-10-30 00:00:00,2020-12-31 00:00:00,NaN
4,805765,A,STA,Texas,TX 147 DENTON,GUYED,ABN,903 South Mayhill Road,NaN,Denton,...,NaN,2020-08-23 00:00:00,2020-04-27 00:00:00,2020-09-07 00:00:00,NaN,2020-06-14 00:00:00,2020-07-04 00:00:00,NaN,NaN,NaN


In [9]:
def assignTeam_specialACQ_Ports(x):
    availableTeams = [df['lightingMaintenance_FOT'][x],
                      df['towerMaintenance'][x],
                      df['TIA_ClimbedInspection'][x],
                      df['towerModificationVendor_or_Inspector'][x]]
    try:
        if min(d for d in availableTeams if isinstance(d, datetime)) <= df['scheduleGBI_Date'][x]:
            if (df['lightingMaintenance_FOT'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'FOT'
            if (df['towerMaintenance'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'TM'
            if (df['TIA_ClimbedInspection'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'TIA'
            if (df['towerModificationVendor_or_Inspector'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'TMV'
        else:
            df['GBI_Team'][x] = 'FOT'
    except:
        df['GBI_Team'][x] = 'FOT'

In [10]:
def assignTeam(x):
    availableTeams = [df['lightingMaintenance_FOT'][x],
                      df['towerMaintenance'][x],
                      df['TIA_ClimbedInspection'][x],
                      df['towerModificationVendor_or_Inspector'][x]]
    try:
        if (df['lightingMaintenance_FOT'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
            df['GBI_Team'][x] = 'FOT'
        if (df['towerMaintenance'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
            df['GBI_Team'][x] = 'TM'
        if (df['TIA_ClimbedInspection'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
            df['GBI_Team'][x] = 'TIA'
        if (df['towerModificationVendor_or_Inspector'][x] == min(d for d in availableTeams if isinstance(d, datetime))):
            df['GBI_Team'][x] = 'TMV'
    except:
        df['GBI_Team'][x] = 'FOT'

In [11]:
for x in range(len(df)):
    if df['inspectMaintainReq'][x] in ['Y', 'P']:
        if df['ACQ_Port'][x] in special_acq_ports:
            assignTeam_specialACQ_Ports(x)
        else:
            assignTeam(x)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\ProgramData\Anac

In [12]:
df.to_excel('siteVisit_output_scenario1.xlsx')

In [ ]:
df['GBI_Team']

In [13]:
visits = df.groupby('GBI_Team').count()

In [14]:
visits

,bu,structureID,area,region,siteName,structType,ACQ_Port,addressLine1,addressLine2,city,...,pre-ConstructionSiteWalk3_CM,pre-ModSiteWalk_CM_or_FOT,towerMaintenance,vegetation1,vegetation2,TIA_ClimbedInspection,constructionGC1,constructionGC2,constructionGC3,towerModificationVendor_or_Inspector
GBI_Team,,,,,,,,,,,,,,,,,,,,,
FOT,27830,27830,27830,27830,27830,27830,26478,27824,407,27829,...,27830,27830,27830,27830,27830,27830,27830,27830,27830,27830
NaN,1208,1208,1208,1208,1208,1208,1071,1182,4,1208,...,1208,1208,1208,1208,1208,1208,1208,1208,1208,1208
TIA,3801,3801,3801,3801,3801,3801,3574,3799,100,3801,...,3801,3801,3801,3801,3801,3801,3801,3801,3801,3801
TM,7020,7020,7020,7020,7020,7020,6430,7018,165,7020,...,7020,7020,7020,7020,7020,7020,7020,7020,7020,7020
TMV,892,892,892,892,892,892,840,892,25,892,...,892,892,892,892,892,892,892,892,892,892


In [ ]:
total_FOT_visits = visits['bu'][0]
total_FOT_visits

In [ ]:
quantity_of_sites_covered_by_vendors = sum(visits['bu'][2:])
quantity_of_sites_covered_by_vendors